In [49]:
import requests
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import os
from scipy.constants import golden

%matplotlib inline

In [2]:
files = ['temp_datalab_records_job_listings.csv', 'temp_datalab_records_job_listings 2.csv']

In [361]:
columns = ['title', 'brand', 'category', 'locality', 'region', 'date_added', 'posted_date']
states = pd.read_csv('us_states.csv', names=['STATE', 'State', 'Abrv'], usecols=['STATE', 'Abrv'])
states_dict = states.set_index('STATE').to_dict()['Abrv']
def _no_location(df):
    truth = np.array(df['region'].isna().tolist() and df['locality'].isna().tolist())
    idx = df[truth].index
    df.drop(idx, inplace=True)
    return None

def _abrv_states(df):
    df['region'] = df['region'].str.upper().replace(states_dict)
    return None

def _in_usa(df):
    truth = df[['region']].isin(states_dict.values())['region']
    idx = df[~truth].index
    df.drop(idx, inplace=True)
    return None

def _combine_dates(df):
    df['posted_date'].fillna(df['date_added'], inplace=True)
    df.drop('date_added', axis=1, inplace=True)
    df.rename(columns={'posted_date': 'date'}, inplace=True)
    return None

def _parse_date(df, columns=['date']):
    for column in columns:
        df[column] = pd.to_datetime(df[column], yearfirst=True)
    return None

def clean_and_save_chunk(file, columns, num=0, chunksize=1e6):
    for chunk in pd.read_csv(file, usecols=columns, chunksize=chunksize):
        _abrv_states(chunk)
        _in_usa(chunk)
        _combine_dates(chunk)
        chunk.reset_index(drop=True).to_feather('data_%i' %num)
        num += 1
    return num

In [362]:
clean_and_save_chunk('temp_datalab_records_job_listings.csv', columns=columns)

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3220: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):
/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3220: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):
